In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tweepy
import pandas as pd
import konlpy
import gensim

# 1) 데이터 준비와 확인

In [2]:
train_data = pd.read_table("./sentimental_classification/data/ratings_train.txt")
test_data = pd.read_table("./sentimental_classification/data/ratings_test.txt")
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 2) 데이터로더 구성

In [6]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']


def load_data(train_data,test_data,num_words=10000):
    #1. 데이터 중복 제거 
    train_data.drop_duplicates(['document'],inplace=True)
    test_data.drop_duplicates(['document'],inplace=True)
    
    #2. Nan 결측치 제거
    train_data = train_data.dropna(how='any') #각 행마다 하나의 결측치라도 있는 행 삭제하기 
    test_data = test_data.dropna(how='any') #각 행마다 하나의 결측치라도 있는 행 삭제하기 
    
    tokenizer = Mecab()
    X_train = []
    for sentence in tqdm(train_data['document']):
        temp_X = tokenizer.morphs(sentence)    
        temp_X = [word for word in temp_X if not word in stopwords]
        X_train.append(temp_X)
        
    X_test = []
    for sentence in tqdm(test_data['document']):
        temp_X = tokenizer.morphs(sentence)    
        temp_X = [word for word in temp_X if not word in stopwords]
        X_train.append(temp_X)
        
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['', '', '', ''] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}
    
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index[''] for word in wordlist]

    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))
    
    return X_train,np.array(list(train_data['label'])),X_test, np.array(list(test_data['label']))
    

In [7]:
from tqdm import tqdm
X_train,y_train,X_test,y_test = load_data(train_data,test_data,num_words=10000)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49157/49157 [00:03<00:00, 12881.28it/s]


### 2-1) 데이터의 중복 제거

In [83]:
train_data.drop_duplicates(['document'],inplace=True)
test_data.drop_duplicates(['document'],inplace=True)

(150000, 3)
(150000, 3)
(146183, 3)


### 2-2) NaN 결측치 제거

In [85]:
test_data.isnull().sum() #결측치를 확인해본 결과 리뷰 document에 5개의 결측치가 존재했다. 
test_data = test_data.dropna(how='any') #각 행마다 하나의 결측치라도 있는 행 삭제하기 
train_data = train_data.dropna(how='any') #각 행마다 하나의 결측치라도 있는 행 삭제하기 
test_data.shape #shape 확인해주기 

(49157, 3)

In [86]:
test_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [87]:
train_x = train_data['document'].to_numpy()
train_y = train_data['label'].to_numpy()
test_x = train_data['document'].to_numpy()
test_y = train_data['label'].to_numpy()

In [88]:
train_x[0]

'아 더빙.. 진짜 짜증나네요 목소리'

### 2-3) 한국어 토크나이저로 토큰화

In [90]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
from tqdm import tqdm

okt = Okt()
kkma = Kkma()

for i in tqdm(range(len(train_x))):
    train_x[i] = okt.morphs(train_x[i])    
print(train_x)

아 더빙.. 진짜 짜증나네요 목소리


100%|██████████████████████████████████| 146182/146182 [14:49<00:00, 164.32it/s]

[list(['아', '더빙', '..', '진짜', '짜증나네요', '목소리'])
 list(['흠', '...', '포스터', '보고', '초딩', '영화', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나'])
 list(['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다']) ...
 list(['이', '게', '뭐', '요', '?', '한국인', '은', '거들', '먹거리', '고', '필리핀', '혼혈', '은', '착하다', '?'])
 list(['청춘', '영화', '의', '최고봉', '.', '방황', '과', '우울했던', '날', '들', '의', '자화상'])
 list(['한국', '영화', '최초', '로', '수간', '하는', '내용', '이', '담긴', '영화'])]


### 2-4) 불용어(Stopwords) 제거

In [92]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']


for i in tqdm(range(len(train_x))):
    train_x[i] = [word for word in train_x[i] if not word in stopwords]

100%|███████████████████████████████| 146182/146182 [00:00<00:00, 147149.70it/s]


In [93]:
print(train_x)

[list(['아', '더빙', '..', '진짜', '짜증나네요', '목소리'])
 list(['흠', '...', '포스터', '보고', '초딩', '영화', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나'])
 list(['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '다']) ...
 list(['게', '뭐', '요', '?', '한국인', '거들', '먹거리', '고', '필리핀', '혼혈', '착하다', '?'])
 list(['청춘', '영화', '최고봉', '.', '방황', '우울했던', '날', '자화상'])
 list(['한국', '영화', '최초', '로', '수간', '하는', '내용', '담긴', '영화'])]


### 2-5) 사전word_to_index 구성

In [110]:
train_x_set = set()
for i in tqdm(range(len(train_x))):
    train_x_set.update(train_x[i])

100%|███████████████████████████████| 146182/146182 [00:00<00:00, 401102.01it/s]

{'스카이폴', '뜰하다', '받칩니다', '볼수만', '산만했던', '유쾌함은', '브뤼주', '27', '약물중독', '많은듯', '하고있네', '찡기', '6~8', 'ニング', '죽이다니', '같으니라', '싱그러웠어요', '오붓한', '조마조마하면서', '했습니까', '찰톤', '조립', '::::', '가져왔으니', '123345677', '괭장히', '사상충', '디저트', '만드시다니', '스즈랑', '섞인', '넘어갔네', '신우', '14:17', '있다니', '마나', '~~~.', '가리네', '멈추진마', '표상', '있는구먼', '사하는데', '쫒을', '대였던것', '셤기간떄', '페그', '번창', '인서트', '호점떄', '지루하다지만', '영향력', '끄는것이', '뻔했지', '재밌었다고하네요', '내리듯이', '버레스크등', 'UP', '송주호', '했었', '...;;;;;;', '처참하게', '힘들었을까', '불가능하다', '음란하거나', '사시미', '열광한던', '건질게', '레오나르도', '행진', '팔짝', '≥∀≤', '추앙', '아니던', '4천', '윙키', '가줄', '되는것이', '깔끔하다', '살아난거', '많음', '마당', '와항', '그만해라', '중요하고', '보다가관둠', '맞다이는', 'Scene', '자라나는', '휘', '바르게', '갈아타는', '오피러스', '용해주어야', '파고들었으요', '욱껴', '평범한거', '넘었는데도', '남기는지', '악플달지', '므슨', '알았다', '휘청', '이래여', '다뎜벼', '왠만해서', '말았다', '카메론', '모셔놓고', '비웃는다', '토기', '세우는것도', '잊고싶은', '7-8년', '평범하나', '뿔', '돋보였었다', '성장할', '현영은', '메이커', '한수', '명확해졌다', '이길말고', '아련히', '재미있었다고', '끼워줘', '은은하게', '알싸하다', '혼주', '애썼어', '벼', '이쁘구요', '감싼다', '떠돌

In [112]:
word_to_index = {}
# 처음 몇 개 인덱스는 사전에 정의되어 있습니다.
word_to_index["<PAD>"] = 0
word_to_index["<BOS>"] = 1
word_to_index["<UNK>"] = 2  # unknown
word_to_index["<UNUSED>"] = 3

In [119]:
#모든원소들의 집합을 만들기 set으로 만들어주기  -> train_x_set
train_x_list = list(train_x_set)
for i in tqdm(range(len(train_x_list))):
    word_to_index[train_x_list[i]] = i+4
    
print(word_to_index['약물중독'])

100%|███████████████████████████████| 104659/104659 [00:00<00:00, 883594.80it/s]

12


### 2-6) 텍스트 스트링을 사전 인덱스 스트링으로 변환

In [123]:
#['아', '더빙', '..', '진짜', '짜증나네요', '목소리'] -> 숫자들로 변환해주기 
raw_inputs = []
for i in range(len(train_x)):
    raw_inputs.append([word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in train_x[i]])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [125]:
print(raw_inputs[0])

[1, 28750, 101157, 84228, 25273, 49240, 35887]


### 2-7) X_train, y_train, X_test, y_test, word_to_index 리턴

In [127]:
print(train_x[0])
print(train_y[0])

['아', '더빙', '..', '진짜', '짜증나네요', '목소리']
0


# 3) 모델 구성을 위한 데이터 분석 및 가공

### 3-1) 데이터셋 내 문장 길이 분포

In [138]:
#X_train,y_train,X_test,y_test = load_data(train_data,test_data,num_words=10000)
total_data_size = X_train + X_test
print(len(total_data_size))

195339


### 3-2) 적절한 최대 문장 길이 지정

### 3-3) keras.preprocessing.sequence.pad_sequences 을 활용한 패딩 추가

# 4) 모델 구성 및 validation set 구성 3가지 이상 구상해보기

In [ ]:
#rnn-> LSTM 사용
#cnn 모델 

# 5) 모델 훈련 개시

# 6) Loss, Accuracy 그래프 시각화

# 7) 학습된 Embedding 레이어 분석

# 8) 한국어 Word2Vec 임베딩 활용하여 성능 개선